In [1]:
import torch

print(torch.__version__)


2.3.1+cpu


In [7]:
import pandas as pd
import numpy as np
import sklearn
import torch
#!pip install transformers
import transformers
#!pip install pytesseract
import pytesseract
#!pip install pdf2image
from pdf2image import convert_from_path
import nltk
#!pip install onnx
import onnx
#!pip install onnxruntime
import onnxruntime as ort

print("All libraries imported successfully!")


  Using cached onnxruntime-1.18.0-cp310-cp310-win_amd64.whl.metadata (4.4 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
  Using cached pyreadline3-3.4.1-py3-none-any.whl.metadata (2.0 kB)
Using cached onnxruntime-1.18.0-cp310-cp310-win_amd64.whl (5.6 MB)
Using cached coloredlogs-15.0.1-py2.py3-none-any.whl (46 kB)
Using cached flatbuffers-24.3.25-py2.py3-none-any.whl (26 kB)
Using cached humanfriendly-10.0-py2.py3-none-any.whl (86 kB)
Using cached pyreadline3-3.4.1-py3-none-any.whl (95 kB)
All libraries imported successfully!


## Convert PDFs to text using OCR:

In [8]:
import pytesseract
from pdf2image import convert_from_path

def pdf_to_text(file_path):
    images = convert_from_path(file_path)
    text = ""
    for image in images:
        text += pytesseract.image_to_string(image)
    return text


##  Data Preprocessing

In [9]:
import re

def annotate_text(text):
    annotations = {}
    annotations['invoice_number'] = re.search(r'Facture Numéro: (\d+)', text).group(1)
    annotations['date'] = re.search(r'Date: (\d{4}-\d{2}-\d{2})', text).group(1)
    annotations['company'] = re.search(r'Entreprise: ([\w\s]+)', text).group(1)
    annotations['amount'] = re.search(r'Montant: (\d+ EUR)', text).group(1)
    return annotations


## Model Training

In [10]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, Trainer, TrainingArguments

model_name = "dbmdz/bert-base-german-cased"
model = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Fine-tune the model with your dataset...


config.json:   0%|          | 0.00/456 [00:00<?, ?B/s]

C:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--dbmdz--bert-base-german-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/240k [00:00<?, ?B/s]

## Model Optimization
## 1. Convert to ONNX

In [11]:
import torch
import onnx

dummy_input = torch.zeros(1, 512, dtype=torch.int64)
torch.onnx.export(model, dummy_input, "model.onnx")


## 2. Optimize the Model
# Use quantization

In [12]:
from onnxruntime.quantization import quantize_dynamic, QuantType

quantize_dynamic("model.onnx", "model_quantized.onnx", weight_type=QuantType.QUInt8)


## 3: Model Deployment

## Write a script to load and run the optimized model

In [13]:
import onnxruntime as ort
import numpy as np

ort_session = ort.InferenceSession("model_quantized.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

def run_model(input_text):
    inputs = tokenizer(input_text, return_tensors="pt")
    ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(inputs['input_ids'])}
    ort_outs = ort_session.run(None, ort_inputs)
    return ort_outs
